# README
The Label Studio output of a annotation task is generated using its own schema which differs than the LOST schema, thus reshaping is required to maintain compatibility with the existing pipeline. The purpose of this notebook is to develop code for postprocessing Label Studio output into a format ready to use for the existing Object Detection Pipeline or fit for use.

The Label Studio tasks export one annotation task (collection of all bounding boxes/labels in the session) per row (assume csv for this description, the equivalent JSON is one list item per annotation task). The expected format is usually one annotation (bounding box/label/etc) per row.

In its simplest form, the CSV/JSON export will contain one row with all the annotations. Our parsing takes into account the scenario that the input file may contain multiple task, even though it may be unlikely.

## Annotations

We want to create a task for identifying the state of a Nest (Incubating, Chick, Fledgeling, Empty). For this purpose, we (prim. Sam) have setup a Label Studio task. The task that Sam has setup can work for her purposes, however, the exported CSV or JSON are not in the format she desires (one nest-per-row), instead the CSV is one annotation task-per-row.

Using the JSON-MIN export, we parse the JSON into the expected CSV.

In [2]:
import json
import pandas as pd
from pathlib import Path 
import time


In [18]:
start_time = time.time()

json_file = Path("~/Downloads/project-29-at-2024-07-06-01-31-8557ce74.json")

print("Reading json file " + str(json_file) + " ...")
# pd.read_json(json_file)
with open(json_file.expanduser(), 'r') as f:
    data = json.load(f)

print("All columns:", list(data[0].keys()))

data[0]


Reading json file ~/Downloads/project-29-at-2024-07-06-01-31-8557ce74.json ...
All columns: ['image', 'id', 'bbox', 'label', 'BeamID', 'NestID', 'annotator', 'annotation_id', 'created_at', 'updated_at', 'lead_time']


{'image': '/data/local-files/?d=samples/images/png/png/20240514_IWMBSpan2_RockyPoint_gigapan_panorama_REDUCED_10.jpg',
 'id': 2499,
 'bbox': [{'x': 43.39152723355047,
   'y': 47.9893135301686,
   'width': 0.874240127988223,
   'height': 1.2547151825914185,
   'rotation': 0,
   'original_width': 5538,
   'original_height': 2558},
  {'x': 41.30906760283686,
   'y': 49.010098763462935,
   'width': 0.28486476080513445,
   'height': 1.5737105679961272,
   'rotation': 0,
   'original_width': 5538,
   'original_height': 2558},
  {'x': 42.20295357639794,
   'y': 49.24402871276002,
   'width': 0.37327106588261216,
   'height': 2.1266359026973305,
   'rotation': 0,
   'original_width': 5538,
   'original_height': 2558},
  {'x': 42.988787399308706,
   'y': 50.49874389535144,
   'width': 0.7072504406196813,
   'height': 1.042051592321691,
   'rotation': 0,
   'original_width': 5538,
   'original_height': 2558},
  {'x': 53.63683569974956,
   'y': 38.14298930067997,
   'width': 0.500969062105618,
  

In [19]:
print("Parsing json file into desired shape...")

def parse_item(data):
    "parse one element from the list of annotation tasks. A json file may contain multiple annotation tasks"
    results = []
    for coords, labels, beamidx, nestidx in zip(data.get('bbox'), data.get('label'), data.get('BeamID'), data.get('NestID')):
        l = labels.get('labels')[0]
        extracted = {'label': l, 'BeamID': beamidx, 'NestID': nestidx}
        extracted.update(coords)

        results.append(extracted)

    # print(json.dumps(results, indent=4)) # printing just to look pretty

    df = pd.json_normalize(results)
    return df

result = []
annot_cols = ['bbox', 'label', 'BeamID', 'NestID'] # columns that need extra parsing attention

for item in data: # in case the json contains multiple annotation tasks, loop through them
    id_cols = [col for col in item.keys() if col not in annot_cols] # remaining columns
    
    df = parse_item(item)
    df[id_cols] = [item.get(c) for c in id_cols] # append columns for id keys

    # resort for readability
    new_cols = [c for c in df.columns if c not in id_cols] # columns produced by expanding and extra parsing
    df = df[id_cols + new_cols] 
    print("Total columns:", df.columns.to_list())
    result.append(df)

df = pd.concat(result)


Parsing json file into desired shape...
Total columns: ['image', 'id', 'annotator', 'annotation_id', 'created_at', 'updated_at', 'lead_time', 'label', 'BeamID', 'NestID', 'x', 'y', 'width', 'height', 'rotation', 'original_width', 'original_height']


In [20]:
print("--- %s seconds ---" % (time.time() - start_time))

print("\nSaving as csv in the same folder as the input file..")
csv_file = json_file.with_suffix(".csv")

df.to_csv(csv_file.expanduser(), index=False)

print(f"Complete. Saved to {csv_file}")

--- 0.0172271728515625 seconds ---

Saving as csv in the same folder as the input file..
Complete. Saved to ~/Downloads/project-29-at-2024-07-06-01-31-8557ce74.csv


In [21]:
df

,image,id,annotator,annotation_id,created_at,updated_at,lead_time,label,BeamID,NestID,x,y,width,height,rotation,original_width,original_height
0,/data/local-files/?d=samples/images/png/png/20...,2499,4,1783,2024-07-05T16:44:57.831405Z,2024-07-05T16:44:57.831443Z,145.251,Incubating Adult,88,40,43.391527,47.989314,0.874240,1.254715,0,5538,2558
1,/data/local-files/?d=samples/images/png/png/20...,2499,4,1783,2024-07-05T16:44:57.831405Z,2024-07-05T16:44:57.831443Z,145.251,Incubating Adult,88,10,41.309068,49.010099,0.284865,1.573711,0,5538,2558
2,/data/local-files/?d=samples/images/png/png/20...,2499,4,1783,2024-07-05T16:44:57.831405Z,2024-07-05T16:44:57.831443Z,145.251,Fledgeling,88,20,42.202954,49.244029,0.373271,2.126636,0,5538,2558
3,/data/local-files/?d=samples/images/png/png/20...,2499,4,1783,2024-07-05T16:44:57.831405Z,2024-07-05T16:44:57.831443Z,145.251,Dead or Empty nest,88,30,42.988787,50.498744,0.707250,1.042052,0,5538,2558
4,/data/local-files/?d=samples/images/png/png/20...,2499,4,1783,2024-07-05T16:44:57.831405Z,2024-07-05T16:44:57.831443Z,145.251,Incubating Adult,999,100,53.636836,38.142989,0.500969,0.999519,0,5538,2558


In [22]:

df

,image,id,annotator,annotation_id,created_at,updated_at,lead_time,label,BeamID,NestID,x,y,width,height,rotation,original_width,original_height
0,/data/local-files/?d=samples/images/png/png/20...,2499,4,1783,2024-07-05T16:44:57.831405Z,2024-07-05T16:44:57.831443Z,145.251,Incubating Adult,88,40,43.391527,47.989314,0.874240,1.254715,0,5538,2558
1,/data/local-files/?d=samples/images/png/png/20...,2499,4,1783,2024-07-05T16:44:57.831405Z,2024-07-05T16:44:57.831443Z,145.251,Incubating Adult,88,10,41.309068,49.010099,0.284865,1.573711,0,5538,2558
2,/data/local-files/?d=samples/images/png/png/20...,2499,4,1783,2024-07-05T16:44:57.831405Z,2024-07-05T16:44:57.831443Z,145.251,Fledgeling,88,20,42.202954,49.244029,0.373271,2.126636,0,5538,2558
3,/data/local-files/?d=samples/images/png/png/20...,2499,4,1783,2024-07-05T16:44:57.831405Z,2024-07-05T16:44:57.831443Z,145.251,Dead or Empty nest,88,30,42.988787,50.498744,0.707250,1.042052,0,5538,2558
4,/data/local-files/?d=samples/images/png/png/20...,2499,4,1783,2024-07-05T16:44:57.831405Z,2024-07-05T16:44:57.831443Z,145.251,Incubating Adult,999,100,53.636836,38.142989,0.500969,0.999519,0,5538,2558


## Masking

Existing masks were annotated using LOST. The masks were used to develop the rest of the pipeline. The Label Studio output of a mask annotation task is generated using a different schema, thus reshaping is required to maintain compatibility with the existing pipeline.

The main differences are:
- columns name 'image' (Label Studio) should be 'img.img_path' (LOST)
- Mask points need to be reshaped
  - Mask points in LOST are a list of dictionary objects with keys 'x' and 'y' and values (float between 0-100). `[{"x": float_x, "y": float_y}, {'x':.., 'y':..}, ...]`
  - Mask points in Label Studio are a list containing a 2-element list containing float between 0-100  `[[float_x, float_y],[..,..], ...]`
  - the column containing the list of mask points must be called `anno.data` (LOST) instead of `points`


Using the CSV export, 

(**Deprecated** code, see section [Masking with JSON-Min](#masking-with-json-min))

In [23]:
start_time = time.time()


In [24]:
csv_file=Path("~/Downloads/project-27-at-2024-07-07-02-01-c58e6a8e.csv")
df = pd.read_csv(csv_file.expanduser(), index_col='annotation_id')

df
d = df['label'].iloc[0]


In [25]:

x = json.loads(d)
print(x)
# x.append(x[0])
dt = pd.json_normalize(x)
dt

[{'points': [[10.066001887581631, 98.94551845342706], [19.969648906008718, 98.76977152899823], [21.4308427283996, 93.49736379613357], [23.541456027408653, 87.69771528998243], [26.301488803035873, 80.31634446397187], [29.46740875154945, 73.81370826010544], [38.23457168589474, 59.40246045694201], [42.2934434147583, 54.13005272407732], [49.9241222650218, 47.803163444639715], [56.09360729289441, 42.1792618629174], [66.3219640496306, 37.609841827768015], [68.10786761033056, 33.21616871704745], [73.87146546531682, 30.579964850615116], [86.61632269394839, 26.18629173989455], [94.81524358625279, 20.56239015817223], [96.4387922777982, 18.27768014059754], [99.8482445300436, 13.884007029876976], [99.76706709546632, 0.35149384885764495], [98.79293788053907, 1.054481546572935], [80.77154740438486, 10.54481546572935], [53.09004221353537, 26.36203866432337], [41.88755624187195, 34.62214411247803], [28.08739236373584, 46.748681898066785], [19.40140686396782, 64.49912126537785], [10.958953667931615, 84

,points,closed,polygonlabels,original_width,original_height
0,"[[10.066001887581631, 98.94551845342706], [19....",True,[IWMB mask],5538,2558


In [3]:

def to_dict(items: pd.Series | list | set, scale=1) -> list:
    """Convert list to [dict]. [float_x, float_y] -> [{"x":float_x, "y":float_y}]"""
    return [{'x': x/scale, 'y': y/scale} for x, y in items]

# df['points'] = df['points'].map(to_dict)
# df

In [27]:
def parse(json_string:str) -> pd.DataFrame:
    """Create dataframe of json object"""
    obj = json.loads(json_string)
    df = pd.json_normalize(obj)
    return df


# create DataFrames of the 'label' object, keep ids for merging later on
results = []
for idx, json_str in zip(df.index, df['label']):
    dt = parse(json_str)
    dt.index = [idx]
    dt.index.name = 'annotation_id'
    
    results.append(dt)

dt = pd.concat(results)

dt['points'] = dt['points'].map(to_dict) # convert list of points (2D array) to dictionary (LOST format)

# Combine and save
df = pd.merge(df, dt, left_index=True, right_index=True)
df = df.rename(columns={
    'points': 'anno.data', 
    'image': 'img.img_path' }) # In the csv output of LOST the columns are called 'anno.data' and 'img.img_path'




In [28]:
print("--- %s seconds ---" % (time.time() - start_time))
print()

print("\nSaving as csv in the same folder as the input file..")
new_name = Path(csv_file).stem + "_parsed_from_csv"
new_name = Path(csv_file).with_suffix('.csv').with_stem(new_name)
df.to_csv(new_name.expanduser())

print(f"Complete. Saved to {new_name}")


--- 0.10197591781616211 seconds ---


Saving as csv in the same folder as the input file..
Complete. Saved to ~/Downloads/project-27-at-2024-07-07-02-01-c58e6a8e_parsed_from_csv.csv


## Masking with JSON-MIN

In order to standardize the code, the CSV parser is ported to use the equivalent JSON-MIN input file. The `parse_item` function changed (and one other constant variables `print`), otherwise the [Annotations](#annotations) JSON-MIN parser is reused.

In [6]:
start_time = time.time()
json_file = Path("~/Downloads/project-27-at-2024-07-07-02-01-c58e6a8e.json")
json_file = Path("~/Downloads/project-27-at-2024-07-13-01-38-1fce07de.json")

def read(json_file):


    print("Reading json file " + str(json_file) + " ...")
# pd.read_json(json_file)
    with open(json_file.expanduser(), 'r') as f:
        data = json.load(f)

    return data

data = read(json_file)
print("All columns:", list(data[0].keys()))
print("inner structure (label):")
data[0].get('label')



Reading json file ~/Downloads/project-27-at-2024-07-13-01-38-1fce07de.json ...
All columns: ['image', 'id', 'label', 'annotator', 'annotation_id', 'created_at', 'updated_at', 'lead_time']
inner structure (label):


[{'points': [[10.331842997915155, 95.44025157232704],
   [16.66669563897262, 78.30188679245283],
   [23.90652722875258, 63.9937106918239],
   [29.939720220235884, 55.503144654088054],
   [36.802477248048135, 47.0125786163522],
   [44.871872874157056, 39.77987421383648],
   [52.2625342887241, 34.43396226415094],
   [63.12228167339404, 26.88679245283019],
   [73.8311992332769, 21.38364779874214],
   [86.72714925257246, 15.251572327044025],
   [97.9639711992101, 11.949685534591195],
   [98.34104576117781, 25.471698113207548],
   [84.31387205597913, 30.81761006289308],
   [73.37870975891565, 35.691823899371066],
   [63.725600972542374, 40.56603773584906],
   [55.65620534643345, 45.911949685534594],
   [47.81305445750516, 52.04402515723271],
   [40.87488251729937, 59.11949685534591],
   [34.841689525816065, 67.45283018867924],
   [26.84770881210069, 80.81761006289308],
   [19.834121959501353, 96.38364779874213]],
  'closed': True,
  'polygonlabels': ['IWMB mask'],
  'original_width': 5427,


In [7]:
print("Parsing json file into desired shape...")

def parse_item(data):
    "parse one element from the list of annotation tasks. A json file may contain multiple annotation tasks"
    
    label = data.get('label')
    label = label[0].copy() # copy() to ensure no in-place modifications
    # print(label)
    
    points = label.get('points')
    points = to_dict(points.copy(), scale=100) 
        # copy() to ensure no in-place modifications
        # scale=100 scales the coordinates by /100
          # LabelStudio units are in the range 0-100; LOST units are 0-1
          # https://labelstud.io/tags/polygonlabels#Sample-Results-JSON
    # print(points)
    
    label.update({'points': points})

    df = pd.json_normalize(label)
    return df

annot_cols = ['label'] # columns that need extra parsing attention
def per_file(data, annot_cols):
    ## TODO: Seems like this function/or downstream is not an idempotent operation.
    
    result = []
    # All Label Studio cols ['image', 'id', 'label', 'annotator', 'annotation_id', 'created_at', 'updated_at', 'lead_time']

    for item in data: # in case the json contains multiple annotation tasks, loop through them
        id_cols = [col for col in item.keys() if col not in annot_cols] # remaining columns
    
        df = parse_item(item)
        df[id_cols] = [item.get(c) for c in id_cols] # append columns for id keys

        # resort for readability
        new_cols = [c for c in df.columns if c not in id_cols] # columns produced by expanding and extra parsing
        df = df[id_cols + new_cols] 
    
        # print("Total columns:", df.columns.to_list())

        result.append(df)
    return result

result = per_file(data, annot_cols)

def combine(result, dtype='polygon'):
    df = pd.concat(result)

    # Label Studio does not have and image_id, using image (path) in its place
    df['img.idx'] = df['image'] 
    
    # renaming to match LOST format
    df = df.rename(columns={
        'id': 'anno.anno_task_id',
        'annotator': 'anno.user_id',
        'points': 'anno.data', 
        'annotation_id': 'anno.lbl.idx',
        'polygonlabels': 'anno.lbl.name',
        'lead_time': 'img.anno_time', # Label Studio does not have 'annotation_time', the closest field is 'lead_time'
        'image': 'img.img_path' }) # In the csv output of LOST the columns are called 'anno.data' and 'img.img_path'

    # In Label Studio, all masks are polygons, unlike LOST (which has a dedicated column)
    df['anno.dtype'] = dtype 
    
    df['anno.lbl.idx'] = df['anno.lbl.idx'].apply(lambda x: [x] if not isinstance(x, list) else x) # the expected data type is list[int]
    
    # The user admin is filtered-out. Label Studio does not provide usernames, so we hack it this way.
    df['img.annotator'] = df['anno.user_id'] 
    df = df.replace({'img.annotator': {0: 'admin'} }) # img.annotator os required, 'anno.user_id' and 'anno.annorator' are not used, but are included for completion
    df['anno.annotator'] = df['img.annotator']

    # TODO: not sure about utility or need of 'anno.lbl.ex' or 'anno.lbl.external_id', which is found commented in the repository code.
    # Instances of anno.lbl.ex are concurrent with anno.lbl.idx are used in a way that it is safe to skip

    # reorder for readability
    lost_cols = ['img.idx', 'img.anno_task_id', 'img.timestamp', 'img.timestamp_lock', 'img.state', 'img.sim_class', 'img.frame_n', 'img.video_path', 'img.img_path', 'img.result_id', 'img.iteration', 'img.user_id', 'img.anno_time', 'img.lbl.idx', 'img.lbl.name', 'img.lbl.external_id', 'img.annotator', 'img.is_junk', 'anno.idx', 'anno.anno_task_id', 'anno.timestamp', 'anno.timestamp_lock', 'anno.state', 'anno.track_id', 'anno.dtype', 'anno.sim_class', 'anno.iteration', 'anno.user_id', 'anno.img_anno_id', 'anno.annotator', 'anno.confidence', 'anno.anno_time', 'anno.lbl.idx', 'anno.lbl.name', 'anno.lbl.external_id', 'anno.data']
    
    # Label Studio columns in order of LOST columns
    new_cols_order = [c for c in lost_cols if c in df.columns.tolist() ]
    # New Label Studio columns not among LOST columns
    drop_cols_order = [c for c in df.columns.tolist() if c not in lost_cols]

    df = df[new_cols_order + drop_cols_order]
    
    return df.reset_index(drop=True)

df = combine(result)



Parsing json file into desired shape...


In [8]:
print("--- %s seconds ---" % (time.time() - start_time))
def save(json_file, df):

    print("\nSaving as csv in the same folder as the input file..")
    new_name = Path(json_file).stem + "_parsed"
    new_name = Path(json_file).with_suffix('.csv').with_stem(new_name)

    df.to_csv(new_name.expanduser(), index=False)
    print(f"Complete. Saved to {new_name}")

save(json_file, df)


--- 0.7763800621032715 seconds ---

Saving as csv in the same folder as the input file..
Complete. Saved to ~/Downloads/project-27-at-2024-07-13-01-38-1fce07de_parsed.csv


In [9]:
df

,img.idx,img.img_path,img.anno_time,img.annotator,anno.anno_task_id,anno.dtype,anno.user_id,anno.annotator,anno.lbl.idx,anno.lbl.name,anno.data,created_at,updated_at,closed,original_width,original_height
0,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,243.367,2,2545,polygon,2,2,[1786],[IWMB mask],"[{'x': 0.10331842997915155, 'y': 0.95440251572...",2024-07-11T22:19:08.276368Z,2024-07-11T22:19:08.276404Z,True,5427,2603
1,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,46.107,2,2546,polygon,2,2,[1787],[IWMB mask],"[{'x': 0.09130324993295094, 'y': 0.89969135802...",2024-07-11T22:20:04.977805Z,2024-07-11T22:20:04.977860Z,True,5708,2791
2,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,60.980,2,2547,polygon,2,2,[1788],[IWMB mask],"[{'x': 0.0890017089204081, 'y': 0.881673881673...",2024-07-11T22:22:00.288703Z,2024-07-11T22:22:00.288741Z,True,5904,3086
3,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,33.702,2,2548,polygon,2,2,[1789],[IWMB mask],"[{'x': 0.09653092006033184, 'y': 0.85869989616...",2024-07-11T22:22:35.923435Z,2024-07-11T22:22:35.923471Z,True,5735,3017
4,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,38.255,2,2549,polygon,2,2,[1790],[IWMB mask],"[{'x': 0.09355893888676692, 'y': 0.92834890965...",2024-07-11T22:23:27.317260Z,2024-07-11T22:23:27.317298Z,True,5799,2809
5,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,28.893,2,2550,polygon,2,2,[1791],[IWMB mask],"[{'x': 0.1033182503770739, 'y': 0.946971439083...",2024-07-11T22:24:00.343078Z,2024-07-11T22:24:00.343129Z,True,5538,2558
6,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,28.702,2,2551,polygon,2,2,[1792],[IWMB mask],"[{'x': 0.1033182503770739, 'y': 0.903864788312...",2024-07-11T22:24:56.807713Z,2024-07-11T22:24:56.807751Z,True,6052,2752
7,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,32.138,2,2552,polygon,2,2,[1793],[IWMB mask],"[{'x': 0.10638141218086827, 'y': 0.81103678929...",2024-07-11T22:25:30.849114Z,2024-07-11T22:25:30.849155Z,True,6155,2777
8,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,35.853,2,2553,polygon,2,2,[1794],[IWMB mask],"[{'x': 0.10708898944193061, 'y': 0.83280848722...",2024-07-11T22:26:08.995898Z,2024-07-11T22:26:08.995936Z,True,6593,2812
9,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,24.717,2,2554,polygon,2,2,[1795],[IWMB mask],"[{'x': 0.08974358974358974, 'y': 0.95082622040...",2024-07-11T22:26:36.242289Z,2024-07-11T22:26:36.242324Z,True,5538,2798


In [10]:
# Rerun all with one cell

json_file = Path("~/Downloads/project-27-at-2024-07-13-01-38-1fce07de.json")

data = read(json_file)
annot_cols = ['label'] # columns that need extra parsing attention
result = per_file(data, annot_cols)
df = combine(result)
save(json_file, df)


Reading json file ~/Downloads/project-27-at-2024-07-13-01-38-1fce07de.json ...

Saving as csv in the same folder as the input file..
Complete. Saved to ~/Downloads/project-27-at-2024-07-13-01-38-1fce07de_parsed.csv


In [11]:
df

,img.idx,img.img_path,img.anno_time,img.annotator,anno.anno_task_id,anno.dtype,anno.user_id,anno.annotator,anno.lbl.idx,anno.lbl.name,anno.data,created_at,updated_at,closed,original_width,original_height
0,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,243.367,2,2545,polygon,2,2,[1786],[IWMB mask],"[{'x': 0.10331842997915155, 'y': 0.95440251572...",2024-07-11T22:19:08.276368Z,2024-07-11T22:19:08.276404Z,True,5427,2603
1,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,46.107,2,2546,polygon,2,2,[1787],[IWMB mask],"[{'x': 0.09130324993295094, 'y': 0.89969135802...",2024-07-11T22:20:04.977805Z,2024-07-11T22:20:04.977860Z,True,5708,2791
2,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,60.980,2,2547,polygon,2,2,[1788],[IWMB mask],"[{'x': 0.0890017089204081, 'y': 0.881673881673...",2024-07-11T22:22:00.288703Z,2024-07-11T22:22:00.288741Z,True,5904,3086
3,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,33.702,2,2548,polygon,2,2,[1789],[IWMB mask],"[{'x': 0.09653092006033184, 'y': 0.85869989616...",2024-07-11T22:22:35.923435Z,2024-07-11T22:22:35.923471Z,True,5735,3017
4,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,38.255,2,2549,polygon,2,2,[1790],[IWMB mask],"[{'x': 0.09355893888676692, 'y': 0.92834890965...",2024-07-11T22:23:27.317260Z,2024-07-11T22:23:27.317298Z,True,5799,2809
5,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,28.893,2,2550,polygon,2,2,[1791],[IWMB mask],"[{'x': 0.1033182503770739, 'y': 0.946971439083...",2024-07-11T22:24:00.343078Z,2024-07-11T22:24:00.343129Z,True,5538,2558
6,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,28.702,2,2551,polygon,2,2,[1792],[IWMB mask],"[{'x': 0.1033182503770739, 'y': 0.903864788312...",2024-07-11T22:24:56.807713Z,2024-07-11T22:24:56.807751Z,True,6052,2752
7,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,32.138,2,2552,polygon,2,2,[1793],[IWMB mask],"[{'x': 0.10638141218086827, 'y': 0.81103678929...",2024-07-11T22:25:30.849114Z,2024-07-11T22:25:30.849155Z,True,6155,2777
8,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,35.853,2,2553,polygon,2,2,[1794],[IWMB mask],"[{'x': 0.10708898944193061, 'y': 0.83280848722...",2024-07-11T22:26:08.995898Z,2024-07-11T22:26:08.995936Z,True,6593,2812
9,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,/data/local-files/?d=2024_IWMB_IWMBSpan2/jpg/2...,24.717,2,2554,polygon,2,2,[1795],[IWMB mask],"[{'x': 0.08974358974358974, 'y': 0.95082622040...",2024-07-11T22:26:36.242289Z,2024-07-11T22:26:36.242324Z,True,5538,2798
